In [72]:
import random

class Sentiment:
    Negative='Negative'
    Neutral='Neutral'
    Positive='Positive'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment=self.get_sentiment()
    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.Negative
        elif self.score ==3:
            return Sentiment.Neutral
        else: 
            return Sentiment.Positive
    
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews=reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative=list(filter(lambda x: x.sentiment==Sentiment.Negative, self.reviews))
        positive=list(filter(lambda x: x.sentiment==Sentiment.Positive, self.reviews))
        positive_shrunk=positive[:len(negative)]
        self.reviews=negative+positive_shrunk
        random.shuffle(self.reviews)
        

In [151]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
file_name='./books_small_10000.json'
#empty list;
reviews= []
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        #print(review['ReviewText'])
        row=Review(review['reviewText'],review['overall'])
        reviews.append(row)
objectreview=reviews[11]
print(type(objectreview))
print(objectreview.sentiment)

<class '__main__.Review'>
Positive


In [129]:
onetwo=list(filter(lambda x: x.score==1|2, reviews))
len(onetwo)
#five=list(filter(lambda x: x.score==5, reviews))
#len(five)

978

In [68]:
negative=list(filter(lambda x: x.sentiment==Sentiment.Negative, reviews))
positive=list(filter(lambda x: x.sentiment==Sentiment.Positive, reviews))
positive_shrunk=positive[:len(negative)]
reviews2=[]
reviews2=negative+positive_shrunk
len(positive_shrunk)
len(reviews2)


1288

In [73]:
from sklearn.model_selection import train_test_split
training, test=train_test_split(reviews, test_size=0.33, random_state=42)
train_container=ReviewContainer(training)
test_container=ReviewContainer(test)
train_container.evenly_distribute()

len(train_container.reviews)

872

In [155]:
train_x=train_container.get_text()
train_y=train_container.get_sentiment()
#train_x=[x.text for x in training]
#train_y=[x.sentiment for x in training]




test_x=test_container.get_text()
test_y=test_container.get_sentiment()

print(train_y.count(Sentiment.Positive))
print(train_y.count(Sentiment.Negative))
print(type(train_y[10]))
print(type(train_y))
print(train_y[1:5])

436
436
<class 'str'>
<class 'list'>
['Positive', 'Negative', 'Negative', 'Positive']


In [156]:
#bag of words vectorization
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#vectorizer=CountVectorizer()
vectorizer=TfidfVectorizer()

train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)
#print(train_x[0])
#print(train_x_vectors[0])
#print(test_x_vectors[0])
print(type(train_x_vectors))

<class 'scipy.sparse.csr.csr_matrix'>


In [97]:
#linear SVM
from sklearn import svm

clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

test_x[0]
#test_x_vectors[0]
clf_svm.predict(test_x_vectors[0])

array(['Positive'], dtype='<U8')

In [99]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)
clf_dec.predict(test_x_vectors[0])

array(['Negative'], dtype='<U8')

In [101]:
from sklearn.neural_network import MLPClassifier
clf_nn= MLPClassifier()
clf_nn.fit(train_x_vectors, train_y)
clf_nn.predict(test_x_vectors[0])

array(['Positive'], dtype='<U8')

In [103]:
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(train_x_vectors, train_y)
clf_log.predict(test_x_vectors[0])

array(['Positive'], dtype='<U8')

In [115]:
#evaluation
clf_svm.score(test_x_vectors, test_y)

0.7412121212121212

In [105]:
clf_dec.score(test_x_vectors, test_y)

0.5803030303030303

In [122]:
clf_log.score(test_x_vectors, test_y)

0.7327272727272728

In [107]:
clf_nn.score(test_x_vectors, test_y)

0.7384848484848485

In [108]:
#F1 score
from sklearn.metrics import f1_score
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.Positive, Sentiment.Neutral, Sentiment.Negative])

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([0.87656461, 0.        , 0.3142329 ])

In [90]:
test_y.count(Sentiment.Negative)

208

In [111]:
test_set=['fun','bad book do not buy','horrible waste of time',' recommended a lot']
new_test=vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['Positive', 'Negative', 'Negative', 'Negative'], dtype='<U8')

In [123]:
#save the model
import pickle
with open('./sentiment_classifier.pkl','wb') as f:
    pickle.dump=(clf_log, f)
    

In [124]:
#open a model
with open ('./sentiment_classifier.pkl','rb') as b:
    unpickler=pickle.Unpickler(b)
    loaded_clf=unpickler.load()
    

EOFError: Ran out of input